In [ ]:
# 1
# carregar todos os dados brutos
import pandas as pd

clientes = pd.read_csv("../data/raw/clientes.csv")
servicos = pd.read_csv("../data/raw/servicos.csv")
atendimentos = pd.read_csv("../data/raw/atendimentos.csv")
nps = pd.read_csv("../data/raw/nps.csv")


In [ ]:
# 2
# converter colunas de data - padronização basica
clientes["data_adesao"] = pd.to_datetime(clientes["data_adesao"])

atendimentos["data_abertura"] = pd.to_datetime(atendimentos["data_abertura"])
atendimentos["data_fechamento"] = pd.to_datetime(atendimentos["data_fechamento"])

nps["data_pesquisa"] = pd.to_datetime(nps["data_pesquisa"])

In [ ]:
# 3
# criar metricas derivadas

# SLA real do atendimento
atendimentos["sla_real_min"] = (
    atendimentos["data_fechamento"] - atendimentos["data_abertura"]
).dt.total_seconds() / 60

In [6]:
# 3
# criar metricas derivadas

# metricas temporais
atendimentos["ano"] = atendimentos["data_abertura"].dt.year
atendimentos["mes"] = atendimentos["data_abertura"].dt.month

In [7]:
# 4
# modelo dimensional > no caso, fato + dimensoes
# dimensao clientes

dim_clientes = clientes[[
    "id_cliente",
    "plano",
    "valor_mensal",
    "cidade",
    "estado",
    "status",
    "data_adesao"
]]

In [8]:
# 4
# 
# dimensão serviços

dim_servicos = servicos[[
    "id_servico",
    "nome_servico",
    "categoria",
    "custo_medio",
    "preco_venda"
]]

In [9]:
# 4
#
# fato atendimentos

fato_atendimentos = atendimentos.merge(
    dim_clientes[["id_cliente", "plano"]],
    on="id_cliente",
    how="left"
).merge(
    dim_servicos[["id_servico", "categoria"]],
    on="id_servico",
    how="left"
)


In [ ]:
# 4
#
# fato atendimentos

fato_atendimentos = atendimentos.merge(
    dim_clientes[["id_cliente", "plano"]],
    on="id_cliente",
    how="left"
).merge(
    dim_servicos[["id_servico", "categoria"]],
    on="id_servico",
    how="left"
)

# fato NPS

fato_nps = nps.merge(
    fato_atendimentos[["id_atendimento", "categoria"]],
    on="id_atendimento",
    how="left"
)

In [11]:
dim_clientes.to_csv("../data/processed/dim_clientes.csv", index=False)
dim_servicos.to_csv("../data/processed/dim_servicos.csv", index=False)
fato_atendimentos.to_csv("../data/processed/fato_atendimentos.csv", index=False)
fato_nps.to_csv("../data/processed/fato_nps.csv", index=False)
